In [ ]:
model.train()
model.eval()

In [27]:
# Copy from feedforward.ipynb
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

import pytorch_lightning as pl
from pytorch_lightning import Trainer
import torch.nn.functional as F

In [28]:
#hyper parameters
input_size = 784 # 28x28
hidden_size = 100
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

In [47]:
class LitNeuralNet(pl.LightningModule):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out

    def training_step(self, batch, batch_idx):
        # training_step defined the train loop.
        # It is independent of forward
        images, labels = batch

        # 100, 1, 28, 28
        # 100, 784
        images = images.reshape(-1, 28*28)

        # forward
        outputs = self(images)
        loss = F.cross_entropy(outputs, labels)

        tensorboard_logs = {'train_loss': loss}
        return {'loss' : loss, 'log': tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        # training_step defined the train loop.
        # It is independent of forward
        images, labels = batch

        # 100, 1, 28, 28
        # 100, 784
        images = images.reshape(-1, 28*28)

        # forward
        outputs = self(images)
        loss = F.cross_entropy(outputs, labels)
        return {'val_loss' : loss}

    def validation_epoch_end(self, val_step_outputs):
        avg_loss = torch.stack([x['val_loss'] for x in val_step_outputs]).mean()
        tnesorboard_logs = {'tavg_val_loss': avg_loss}
        return {'val_loss': avg_loss, 'log': tnesorboard_logs}

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(),lr=learning_rate)
        return optimizer
    
    def train_dataloader(self):
        train_dataset = torchvision.datasets.MNIST(root='./data',
                                                    train=True,
                                                    transform=transforms.ToTensor(),
                                                    download=True)

        train_loader = torch.utils.data.DataLoader(train_dataset,
                                                    batch_size=batch_size,
                                                    num_workers=8,
                                                    shuffle=True)
        return train_loader
    
    def val_dataloader(self):
        val_dataset = torchvision.datasets.MNIST(root='./data',
                                                train=False,
                                                transform=transforms.ToTensor())

        val_loader = torch.utils.data.DataLoader(val_dataset,
                                                batch_size=batch_size,
                                                num_workers=4,
                                                shuffle=False)
        return val_loader



In [48]:
trainer = Trainer(deterministic=True, auto_lr_find=True, gpus=1, max_epochs=num_epochs, fast_dev_run=False)
model = LitNeuralNet(input_size, hidden_size, num_classes)
trainer.fit(model)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name | Type   | Params
--------------------------------
0 | l1   | Linear | 78.5 K
1 | relu | ReLU   | 0     
2 | l2   | Linear | 1.0 K 
--------------------------------
79.5 K    Trainable params
0         Non-trainable params
79.5 K    Total params
0.318     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

1

In [49]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

Launching TensorBoard...